In [3]:
import json
import os

restaurant_name = "chianti"
restaurant_file = f"raw_data/{restaurant_name}.json"
image_dir = f"image_data/{restaurant_name}"
processed_dir = f"../processed/{restaurant_name}.json"

def determine_group_category(category):
    if category in ["Pasta", "Insalata (Salads)", "Zuppa (Soup)", "Risotto (Italian Rice)", "Antipasti (Appetizers)", "Garlic Bread"]:
        return "Pasta"
    elif category in ["Pizza", "Vegan"]:
        return "Main Course"
    elif category in ["Insalata (Salads)"]:
        return "Main Course"
    elif category in ["Zuppa (Soup)"]:
        return "Main Course"
    elif category in ["Risotto (Italian Rice)"]:
        return "Main Course"
    elif category in ["Antipasti (Appetizers)"]:
        return "Appetizer"
    elif category in ["Garlic Bread"]:
        return "Appetizer"
    else:
        raise ValueError(f"Unknown category: {category}")

def brief_category(category):
    if category in ["Pasta", "Insalata (Salads)", "Zuppa (Soup)", "Risotto (Italian Rice)", "Antipasti (Appetizers)", "Garlic Bread"]:
        return "Pasta"
    elif category in ["Pizza", "Vegan"]:
        return "Main Course"
    elif category in ["Insalata (Salads)"]:
        return "Main Course"
    elif category in ["Zuppa (Soup)"]:
        return "Main Course"
    elif category in ["Risotto (Italian Rice)"]:
        return "Main Course"
    elif category in ["Antipasti (Appetizers)"]:
        return "Appetizer"
    elif category in ["Garlic Bread"]:
        return "Appetizer"
    else:
        raise ValueError(f"Unknown category: {category}")




In [5]:
# with open(restaurant_file, "r") as f:
#     restaurant_data = json.load(f)

# categories = set()
# for item in restaurant_data:
#     categories.add(item["category_2"])
# categories

mapping_categories = {'Aerated Beverages': {'group_category': 'Drinks', 'category_brief': 'Soda'},
 'Antipasti (Appetizers)': {'group_category': 'Appetizers', 'category_brief': 'Appetizer'},
 'Carne (Meat Dishes)': {'group_category': 'Main Course', 'category_brief': 'Meat'},
 'Cold Pressed Juices': {'group_category': 'Drinks', 'category_brief': 'Juice'},
 'Combos': {'group_category': 'Combo', 'category_brief': 'Combo'},
 'Dolci (Dessert)': {'group_category': 'Dessert', 'category_brief': 'Dessert'},
 'Garlic Bread': {'group_category': 'Appetizers', 'category_brief': 'Garlic Bread'},
 'Insalata (Salads)': {'group_category': 'Main Course', 'category_brief': 'Salad'},
 'Mocktails': {'group_category': 'Drinks', 'category_brief': 'Mocktail'},
 'Pasta': {'group_category': 'Main Course', 'category_brief': 'Pasta'},
 'Pesce (Seafood)': {'group_category': 'Main Course', 'category_brief': 'Seafood'},
 'Piatti Al Forno (Baked Dishes)': {'group_category': 'Main Course', 'category_brief': 'Baked Dish'},
 'Piatto (Platters)': {'group_category': 'Main Course', 'category_brief': 'Platter'},
 'Pizza': {'group_category': 'Main Course', 'category_brief': 'Pizza'},
 'Risotto (Italian Rice)': {'group_category': 'Main Course', 'category_brief': 'Risotto'},
 'Special Shakes': {'group_category': 'Drinks', 'category_brief': 'Shake'},
 'Vegan': {'group_category': 'Main Course', 'category_brief': 'Vegan'},
 'Zuppa (Soup)': {'group_category': 'Appetizers', 'category_brief': 'Soup'}}


In [4]:
with open(restaurant_file, "r") as f:
    restaurant_data = json.load(f)

processed_data = []

counter = iter(range(100000))
for item in restaurant_data:
    image_path = os.path.join(image_dir, item['name'].replace(' ', '_').replace('/', '_') + '.png') if item['image_url'] else None
    category_data = mapping_categories.get(item["category_2"])
    processed_data.append({
        "id": next(counter),
        "name": item["name"],
        "category": item["category_2"],
        "group_category": category_data["group_category"],
        "category_brief": category_data["category_brief"],
        "description": item["description"],
        "price": item["price"],
        "veg_flag": item["is_vegetarian"],
        "image_path": image_path,
        "is_bestseller": item["is_bestseller"] if "is_bestseller" in item else 0,
        "is_chef_recommended": item["is_chef_recommended"] if "is_chef_recommended" in item else 0,
    })

with open(processed_dir, "w") as f:
    json.dump(processed_data, f)

In [3]:
featured = [
    (41, "DHLa1f5ox63"),
    (2 ,"DC6zcioyNnh")
]

In [4]:
with open(processed_dir, "r") as f:
    restaurant_data = json.load(f)

for item in restaurant_data:
    item["featured"] = False
    item["insta_id"] = None

for item_id, insta_id in featured:
    for item in restaurant_data:
        if item["id"] == item_id:
            item["featured"] = True
            item["insta_id"] = insta_id
            break
    
with open(processed_dir, "w") as f:
    json.dump(restaurant_data, f)
    


In [ ]:
import random
import json

with open(processed_dir, "r") as f:
    restaurant_data = json.load(f)

categories = set()
for category in mapping_categories.values():
    if category["group_category"] != "Combo":
        categories.add(category["group_category"])

# For each category, randomly select 3 veg and 3 non-veg items and set is_high_margin=1
for category in categories:
    veg_items = [item for item in restaurant_data if item["group_category"] == category and item["veg_flag"] == 1]
    nonveg_items = [item for item in restaurant_data if item["group_category"] == category and item["veg_flag"] == 0]

    selected_veg = random.sample(veg_items, min(3, len(veg_items)))
    selected_nonveg = random.sample(nonveg_items, min(3, len(nonveg_items)))

    for item in selected_veg + selected_nonveg:
        item["is_high_margin"] = 1

for item in restaurant_data:
    if "is_high_margin" not in item:
        item["is_high_margin"] = 0

# Save the updated data back to the JSON file
with open(processed_dir, "w") as f:
    json.dump(restaurant_data, f, indent=4)